In [1]:
import sys

sys.path.append('../')

from exam_alignment.utils.alignment_utils import one_file_per_process
from exam_alignment.type_exam_parser.abstract_exam_parser import AbstractExamParser
from exam_alignment.type_exam_parser.annotated_exam_parser import AnnotatedExamParser
from exam_alignment.type_exam_parser.standard_exam_parser import StandardExamParser

from exam_alignment.exam_parser_container import ExamParserContainer

from exam_alignment.utils.new_alignment_utils import longest_increasing_subsequence_index

In [2]:
import os
import glob
from pathlib import Path
import re

examination_paper_list = []

path = Path("./docx_markdowns")

for file in path.glob("*.md"):
    with open(file, "r", encoding="utf-8") as f:
        examination_paper_list.append({
            "file_path": str(file),
            "text": one_file_per_process(f.read())
        })
len(examination_paper_list)

2230

In [3]:
remaining_exams = []
no_answer_exams = []

standard_exams = []
annotated_exams = []

for exam_info in examination_paper_list:
    if not AbstractExamParser.check_contains_answers(exam_info["text"]):
        no_answer_exams.append(exam_info)
        continue

    if StandardExamParser.detect_this_exam_type(exam_info["text"]):
        standard_exams.append(exam_info)
        continue

    if AnnotatedExamParser.detect_this_exam_type(exam_info["text"]):
        annotated_exams.append(exam_info)
        continue

    remaining_exams.append(exam_info)

In [4]:
# 识别标注和解析类型的占比
percentage = (len(standard_exams) + len(annotated_exams)) / (len(examination_paper_list)-len(no_answer_exams)) * 100
print(percentage)
# 无答案的试卷
len(remaining_exams)

54.56905503634476


875

In [5]:
len(examination_paper_list)/len(no_answer_exams)

7.3355263157894735

In [6]:
test_text = standard_exams[100]["text"]
print(test_text)


2020-2021学年福建省福州市鼓楼区五年级（上）期末数学试卷
一.填空题。
1．2300cm^2^＝[ ]{.underline}dm^2^
2时45分＝[ ]{.underline}时
2．在横线里填上"＞"、"＝"或"＜"。
  ---------------------------------- ----------------------------------- ----------------------------------- ----------------------------
  5.35÷0.99[ ]{.underline}5.35   2.7÷0.18[ ]{.underline}27÷1.8   8.6÷1.1[ ]{.underline}1.1×8.6   0.0[ ]{.underline}0.06
  ---------------------------------- ----------------------------------- ----------------------------------- ----------------------------
3．每盒饼干6.9元钱，50元最多可以买[ ]{.underline}盒饼干．
4．一个灯塔上的信号灯，闪5下用了20秒，信号灯闪了12下用时[ ]{.underline}秒，30秒最多闪[ ]{.underline}下。
5．光年是计量天体距离的单位，1光年约9.5万亿千米．距地球最近的一颗恒星约有4.22光年，合[ ]{.underline}万亿千米．
6．袋子里有三种大小相同颜色不同的小球，其中红球5个，黑球3个，蓝球2个。任意摸出1个球，摸到[ ]{.underline}球的可能性最大。
7．小强走0.25千米需要0.1小时，他每小时走[ ]{.underline}千米，每走1千米需要[ ]{.underline}小时．
8．一本书有a页，张华每天看5页，看了b天。5b表示[ ]{.underline}，a﹣5b表示[ ]{.underline}。
9．第一颗人造卫星绕地球一圈要用114分钟，比"神舟"六号绕地球一圈所用时间的2倍少66分钟。设"神舟"六号绕地球一圈的时间为x分钟，列方程为[ ]{.underline}。
10．一条直线上有A、B、C三

In [7]:
StandardExamParser(test_text).extract_questions()

['1．2300cm^2^＝[ ]{.underline}dm^2^\n2时45分＝[ ]{.underline}时',
 '2．在横线里填上"＞"、"＝"或"＜"。\n  ---------------------------------- ----------------------------------- ----------------------------------- ----------------------------\n  5.35÷0.99[ ]{.underline}5.35   2.7÷0.18[ ]{.underline}27÷1.8   8.6÷1.1[ ]{.underline}1.1×8.6   0.0[ ]{.underline}0.06\n  ---------------------------------- ----------------------------------- ----------------------------------- ----------------------------',
 '3．每盒饼干6.9元钱，50元最多可以买[ ]{.underline}盒饼干．',
 '4．一个灯塔上的信号灯，闪5下用了20秒，信号灯闪了12下用时[ ]{.underline}秒，30秒最多闪[ ]{.underline}下。',
 '5．光年是计量天体距离的单位，1光年约9.5万亿千米．距地球最近的一颗恒星约有4.22光年，合[ ]{.underline}万亿千米．',
 '6．袋子里有三种大小相同颜色不同的小球，其中红球5个，黑球3个，蓝球2个。任意摸出1个球，摸到[ ]{.underline}球的可能性最大。',
 '7．小强走0.25千米需要0.1小时，他每小时走[ ]{.underline}千米，每走1千米需要[ ]{.underline}小时．',
 '8．一本书有a页，张华每天看5页，看了b天。5b表示[ ]{.underline}，a﹣5b表示[ ]{.underline}。',
 '9．第一颗人造卫星绕地球一圈要用114分钟，比"神舟"六号绕地球一圈所用时间的2倍少66分钟。设"神舟"六号绕地球一圈的时间为x分钟，列方程为[ ]{.underline}。',
 '10．一条直线上有A、B、

In [8]:
StandardExamParser(test_text).extract_answers()

1．【分析】（1）低级单位平方厘米化高级单位平方分米除以进率100；（2）把45分除以进率60化成0.75时再加2时。【解答】解：（1）2300cm^2^＝23dm^2^（2）2时45分＝
2.75时。故答案为：23，2.75。【点评】本题是考查时间的单位换算、面积的单位换算。单位换算首先要弄清是由高级单位化低级单位还是由低级单位化高级单位，其次记住单位间的进率。2．【分析】一个数（0除外）除以小于1的数，商大于这个数；在除法算式中，被除数和除数同时乘或除以相同的数（0除外），商不变；一个数（0除外）除以大于1的数，商小于这个数；一个数（0除外）乘大于1的数，积大于这个数；0.00.0656566，0.060.065556，所以0.00.06。【解答】解：  ----------------- ------------------ ------------------ ---------  5.35÷0.99＞5.35   2.7÷0.18＝27÷1.8   8.6÷1.1＜1.1×8.6   0.00.06  ----------------- ------------------ ------------------ ---------故答案为：＞，＝，＜，＞。【点评】此题主要考查了不用计算判断商与被除数之间大小关系的方法。
3．【分析】求50元最多可以买几盒饼干，即求50里面含有几个6.9，根据求一个数里面含有几个另一个数，用除法解答．【解答】解：50÷6.9≈7（盒）；答：50元最多可以买7盒饼干；故答案为：7．【点评】解答此题应根据求一个数里面含有几个另一个数，用除法解答；注意：应结合实际情况，用"去尾"法．
4．【分析】根据植树问题原理，信号灯闪5下，有5﹣1＝4（个）间隔，闪12下，有12﹣1＝11（个）间隔，用20÷4＝5（秒）求一个间隔的时间，再乘11，得11个间隔的时间；30除以每个间隔的时间，再加上1，就是闪的下数。【解答】解：20÷（5﹣1）＝20÷4＝5（秒）（12﹣1）×5＝11×5＝55（秒）30÷5+1＝6+1＝7（下）答：信号灯闪了12下用时55秒，30秒最多闪7下。故答案为：55；7。【点评】本题主要考查植树问题，关键是利用间隔数与闪的下数的关系做题。
5.35÷0.99＞5.35   2.7÷0.18＝27÷1.8   8.6÷

['1．【分析】（1）低级单位平方厘米化高级单位平方分米除以进率100；（2）把45分除以进率60化成0.75时再加2时。【解答】解：（1）2300cm^2^＝23dm^2^（2）2时45分＝',
 '2.75时。故答案为：23，2.75。【点评】本题是考查时间的单位换算、面积的单位换算。单位换算首先要弄清是由高级单位化低级单位还是由低级单位化高级单位，其次记住单位间的进率。2．【分析】一个数（0除外）除以小于1的数，商大于这个数；在除法算式中，被除数和除数同时乘或除以相同的数（0除外），商不变；一个数（0除外）除以大于1的数，商小于这个数；一个数（0除外）乘大于1的数，积大于这个数；0.00.0656566，0.060.065556，所以0.00.06。【解答】解：  ----------------- ------------------ ------------------ ---------  5.35÷0.99＞5.35   2.7÷0.18＝27÷1.8   8.6÷1.1＜1.1×8.6   0.00.06  ----------------- ------------------ ------------------ ---------故答案为：＞，＝，＜，＞。【点评】此题主要考查了不用计算判断商与被除数之间大小关系的方法。',
 '3．【分析】求50元最多可以买几盒饼干，即求50里面含有几个6.9，根据求一个数里面含有几个另一个数，用除法解答．【解答】解：50÷6.9≈7（盒）；答：50元最多可以买7盒饼干；故答案为：7．【点评】解答此题应根据求一个数里面含有几个另一个数，用除法解答；注意：应结合实际情况，用"去尾"法．',
 '4．【分析】根据植树问题原理，信号灯闪5下，有5﹣1＝4（个）间隔，闪12下，有12﹣1＝11（个）间隔，用20÷4＝5（秒）求一个间隔的时间，再乘11，得11个间隔的时间；30除以每个间隔的时间，再加上1，就是闪的下数。【解答】解：20÷（5﹣1）＝20÷4＝5（秒）（12﹣1）×5＝11×5＝55（秒）30÷5+1＝6+1＝7（下）答：信号灯闪了12下用时55秒，30秒最多闪7下。故答案为：55；7。【点评】本题主要考查植树问题，关键是利用间隔数与闪的下数的关系做题。',
 '5.35÷0.99＞5.35   2.7÷

In [9]:
print(test_text.splitlines()[4])

2．在横线里填上"＞"、"＝"或"＜"。


In [10]:
StandardExamParser.detect_this_exam_type(test_text)

True